In [13]:
import numpy as np
import os
import matplotlib.pyplot as plt
import torchvision
import cv2

In [32]:
a = np.load('inputs/train_images_20190915/img_002be4f.npy')
b = np.load('inputs/train_images_mask_20190915/msk_0011165.npy')

In [35]:
print(a.shape)
a = a.reshape(1, a.shape[0], a.shape[1])
print(a.shape)
a = a.transpose(1,2,0)
print(a.shape)
a = cv2.resize(a, (212,212), interpolation = cv2.INTER_CUBIC)

(525, 325, 1)
(1, 525, 325)
(525, 325, 4)


(212, 212)

In [171]:
c.reshape([525,525]).shape

(525, 525)

In [172]:

cv2.resize(c.reshape([525,525]),(96,96),interpolation = cv2.INTER_CUBIC)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [158]:
cv2.fr

(1, 525, 525)

In [89]:
a.T.shape

(325, 525)

In [58]:
d = random_crop(a)

In [61]:
d.max()

0.0037639486687019125